# Processing data

Read data already collected

In [17]:
import json
import time
import datetime
import pandas as pd
import numpy as np

tweets_data_morn = '/Users/Madhu/Documents/Courses/data_bds/bitcoin'
tweets_data_eve = '/Users/Madhu/Documents/Courses/data_bds/bitcoin2'

tweets_data_m = []
tweets_data_e = []

tweets_file = open(tweets_data_morn, "r")
for line in tweets_file:
    try:
        tweet = json.loads(line)
        tweets_data_m.append(tweet)
    except:
        continue
        
tweets_file = open(tweets_data_eve, "r")
for line in tweets_file:
    try:
        tweet = json.loads(line)
        tweets_data_e.append(tweet)
    except:
        continue


In [18]:
print (len(tweets_data_m))

8035


In [19]:
print (len(tweets_data_e))

23055


In [22]:
tweets_data_e[0]['created_at']

'Fri Oct 20 18:07:48 +0000 2017'

In [29]:
date_col = []
lang = []
text = []

for line in tweets_data_e:
    try:
        date_col.append(datetime.datetime.strptime(line['created_at'],'%a %b %d %H:%M:%S +0000 %Y'))
        lang.append(line['lang'])
        text.append(line['text'])
    except:
        continue

In [33]:
d = {'date': date_col, 'lang': lang, 'text':text}
df = pd.DataFrame(data=d)

In [34]:
df.head()

,date,lang,text
0,2017-10-20 18:07:48,es,RT @CryptoLionTeam: $PTOY #PTOY / #BTC https:/...
1,2017-10-20 18:07:50,tr,Sınav sistemi açıklandıktan 10 dakika sonra fo...
2,2017-10-20 18:07:50,und,La Lingua neo-borbonica https://t.co/lPTUIQuVN7
3,2017-10-20 18:07:51,tr,RT @FelsefeYks: #FelsefesizOlmaz felsefe sınav...
4,2017-10-20 18:07:51,es,@UPYD @ivanrodig Los refugiados de mediterráne...


In [36]:
df_en = df[df['lang']=='en']

In [42]:
pd.set_option('max_colwidth',150)
df_en['text']

5                           #dellservice_eth dellservice.eth listed for 100 ETH by honeyfat2907@gmail.com at #enslisting. Bid now @ https://t.co/EfkWKw7Xhx
6                                                         like we understand that he the first brown boy to get it poppin... lmao 😂 https://t.co/0iPcJhQ4J1
9                       Confideal Announces New Partnership with https://t.co/mp5NIDv6ze https://t.co/2zTg3n0eg6 #bitcoin #bitcoinnews #btc #cryptocurrency
12                                                                                     Subscribe Neo News Web: Like Us: Twitter:... https://t.co/lCCvszVKFj
13               RT @RachelMaxi: Three Neo-Nazis Were Just Arrested After Opening Fire On Protesters At Richard Spencer UF Speech … https://t.co/47xGkzUZhN
14                            Exchanges\na/ Bitmex (leveraged $BTC trading): https://t.co/mfjCRFKHGu\nb/ 1broker (copy trade $BTC): https://t.co/MrUqSLUWPp
17                                                              

In [43]:
df_en.to_csv('bitcoin_eng.csv') # this has all 4 bitcoins
#get data from the csv directly instead of running it till now

## Price of BTC

In [51]:
bitcoin_price = pd.read_csv('/Users/Madhu/Documents/Courses/data_bds/BTC_20.csv')

In [53]:
bitcoin_price.head()

,Date,Close Price
0,10/20/17 0:00,5699.58
1,10/20/17 0:01,5699.40
2,10/20/17 0:02,5700.21
3,10/20/17 0:03,5700.17
4,10/20/17 0:04,5701.31


Sample of grouping using date and count

In [ ]:
grp = data.groupby(by=[data.datetime_col.map(lambda x : (x.hour, x.minute))])
grp.count()